In [9]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create a table to store data from multiple sources
cursor.execute('''
CREATE TABLE IF NOT EXISTS calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER NOT NULL,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    week_number INTEGER NOT NULL,
    quarter INTEGER NOT NULL,
    day_of_week TEXT NOT NULL,
    is_weekend BOOLEAN NOT NULL,
    full_date DATETIME NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')



# Commit changes and close the connection
conn.commit()
conn.close()

print("Database and table created successfully.")

Database and table created successfully.


In [10]:
from datetime import datetime, timedelta

# Generate records from 1900 to 2030
start_date = datetime(1900, 1, 1)
end_date = datetime(2030, 12, 31)
current_date = start_date

records = []
while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    week_number = current_date.isocalendar()[1]
    quarter = (month - 1) // 3 + 1
    day_of_week = current_date.strftime('%A')
    is_weekend = day_of_week in ['Saturday', 'Sunday']
    full_date = current_date.strftime('%Y-%m-%d')
    
    records.append((year, month, day, week_number, quarter, day_of_week, is_weekend, full_date))
    current_date += timedelta(days=1)

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Insert records into the calendar table
cursor.executemany('''
INSERT INTO calendar (year, month, day, week_number, quarter, day_of_week, is_weekend, full_date)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', records)

# Commit changes
conn.commit()

# Close the connection after the operation
conn.close()

print(f"{len(records)} records inserted into the calendar table.")

47847 records inserted into the calendar table.


In [12]:
import fastf1

# Enable the cache for fastf1
import os

# Ensure the cache directory exists
cache_dir = 'fastf1_cache'
os.makedirs(cache_dir, exist_ok=True)

# Enable the cache for fastf1
fastf1.Cache.enable_cache(cache_dir)

# Initialize an empty list to store F1 calendar data
f1_calendar_data = []

# Loop through the years from 2000 to 2025
for year in range(2000, 2026):
    # Load the F1 schedule for the current year
    schedule = fastf1.get_event_schedule(year)
    
    # Extract relevant data for the table
    for _, event in schedule.iterrows():
        race_name = event['EventName']
        country = event['Country']
        circuit = event['Location']
        event_year = event['EventDate'].year
        date = event['EventDate'].strftime('%Y-%m-%d')
        f1_calendar_data.append((race_name, country, circuit, event_year, date))

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the f1_calendar table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    race_name TEXT NOT NULL,
    country TEXT NOT NULL,
    circuit TEXT NOT NULL,
    year INTEGER NOT NULL,
    date TEXT NOT NULL
)
''')

# Insert F1 calendar data into the table
cursor.executemany('''
INSERT INTO fone_calendar (race_name, country, circuit, year, date)
VALUES (?, ?, ?, ?, ?)
''', f1_calendar_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("F1 calendar data from 2000 to 2025 inserted into the database successfully.")


F1 calendar data from 2000 to 2025 inserted into the database successfully.


In [13]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the fone_geography table
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_geography (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    circuit TEXT NOT NULL,
    city TEXT NOT NULL,
    country TEXT NOT NULL,
    continent TEXT NOT NULL,
    latitude REAL NOT NULL,
    longitude REAL NOT NULL,
    existing INTEGER DEFAULT 0
)
''')

# Define a mapping of circuits to their geographic information
# Define map for same circuits
same_circuits = {
    'Monte Carlo': ['Monaco', 'Monte Carlo','Monte-Carlo'],
    'Marina Bay': ['Singapore', 'Marina Bay'],
    'Catalunya': ['Montmeló', 'Barcelona','Spain'],
    'Spa-Francorchamps': ['Spa', 'Stavelot','Spa-Francorchamps'],
    'Montreal': ['Montréal', 'Montreal'],
    'Nürburgring': ['Nürburg', 'Nürburgring'],
    'Yas Marina': ['Yas Island', 'Abu Dhabi', 'Yas Marina'],
    'Sakhir': ['Bahrain', 'Sakhir']
    }
geography_data = [
    ('Silverstone', 'Silverstone', 'United Kingdom', 'Europe', 52.0786, -1.0169, 1),
    ('Budapest', 'Mogyoród', 'Hungary', 'Europe', 47.5789, 19.2486, 1),
    ('Monza', 'Monza', 'Italy', 'Europe', 45.6156, 9.2811, 1),
    ('São Paulo', 'São Paulo', 'Brazil', 'South America', -23.7036, -46.6997, 1),
    ('Sakhir', 'Sakhir', 'Bahrain', 'Asia', 26.0325, 50.5106, 1),
    ('Melbourne', 'Melbourne', 'Australia', 'Oceania', -37.8497, 144.9680, 1),
    ('Suzuka', 'Suzuka', 'Japan', 'Asia', 34.8431, 136.5419, 1),
    ('Shanghai', 'Shanghai', 'China', 'Asia', 31.3389, 121.2197, 1),
    ('Kuala Lumpur', 'Sepang', 'Malaysia', 'Asia', 2.7608, 101.7386, 1),
    ('Catalunya', 'Montmeló', 'Spain', 'Europe', 41.5700, 2.2611, 1),
    ('Spielberg', 'Spielberg', 'Austria', 'Europe', 47.2196, 14.7649, 1),
    ('Montreal', 'Montreal', 'Canada', 'North America', 45.5000, -73.5228, 1),
    ('Hockenheim', 'Hockenheim', 'Germany', 'Europe', 49.3278, 8.5658, 1),
    ('Marina Bay', 'Singapore', 'Singapore', 'Asia', 1.2914, 103.8645, 1),
    ('Austin', 'Austin', 'United States', 'North America', 30.1328, -97.6411, 1),
    ('Imola', 'Imola', 'Italy', 'Europe', 44.3439, 11.7167, 1),
    ('Mexico City', 'Mexico City', 'Mexico', 'North America', 19.4042, -99.0907, 1),
    ('Baku', 'Baku', 'Azerbaijan', 'Asia', 40.3725, 49.8533, 1),
    ('Istanbul', 'Istanbul', 'Turkey', 'Asia', 40.9517, 29.4050, 1),
    ('Magny Cours', 'Magny-Cours', 'France', 'Europe', 46.8642, 3.1633, 1),
    ('Spa-Francorchamps', 'Stavelot', 'Belgium', 'Europe', 50.4379, 5.9714, 1),
    ('Sochi', 'Sochi', 'Russia', 'Europe', 43.4057, 39.9578, 1),
    ('Indianapolis', 'Indianapolis', 'United States', 'North America', 39.7950, -86.2347, 1),
    ('Jeddah', 'Jeddah', 'Saudi Arabia', 'Asia', 21.6319, 39.1044, 1),
    ('Zandvoort', 'Zandvoort', 'Netherlands', 'Europe', 52.3888, 4.5409, 1),
    ('Valencia', 'Valencia', 'Spain', 'Europe', 39.4589, -0.3317, 1),
    ('Le Castellet', 'Le Castellet', 'France', 'Europe', 43.2500, 5.7917, 1),
    ('Lusail', 'Lusail', 'Qatar', 'Asia', 25.4914, 51.4542, 1),
    ('Yeongam County', 'Yeongam', 'South Korea', 'Asia', 34.7333, 126.4170, 1),
    ('Miami', 'Miami', 'United States', 'North America', 25.9581, -80.2389, 1),
    ('Uttar Pradesh', 'Greater Noida', 'India', 'Asia', 28.3500, 77.5350, 1),
    ('Monte Carlo', 'Monte Carlo', 'Monaco', 'Europe', 43.7347, 7.4206, 1),
    ('Las Vegas', 'Las Vegas', 'United States', 'North America', 36.1699, -115.1398, 1),
    ('Portimão', 'Portimão', 'Portugal', 'Europe', 37.2278, -8.6267, 1),
    ('Oyama', 'Oyama', 'Japan', 'Asia', 35.3711, 138.9278, 1),
    ('Nürburgring', 'Nürburg', 'Germany', 'Europe', 50.3356, 6.9475, 1),
    ('Yas Marina', 'Abu Dhabi', 'United Arab Emirates', 'Asia', 24.4672, 54.6031, 1),
    ('Mugello', 'Scarperia e San Piero', 'Italy', 'Europe', 43.9975, 11.3725, 1),
    ('Cape Town', 'Cape Town', 'South Africa', 'Africa', -33.9180, 18.4233, 0),
    ('Seoul', 'Seoul', 'South Korea', 'Asia', 37.5665, 126.9780, 0),
    ('Buenos Aires', 'Buenos Aires', 'Argentina', 'South America', -34.6037, -58.3816, 0),
    ('Chicago', 'Chicago', 'United States', 'North America', 41.8781, -87.6298, 0),
    ('Auckland', 'Auckland', 'New Zealand', 'Oceania', -36.8485, 174.7633, 0),
    ('Riyadh', 'Riyadh', 'Saudi Arabia', 'Asia', 24.7136, 46.6753, 0),
    ('Lagos', 'Lagos', 'Nigeria', 'Africa', 6.5244, 3.3792, 0),
    ('Oslo', 'Oslo', 'Norway', 'Europe', 59.9139, 10.7522, 0),
    ('Kuala Terengganu', 'Kuala Terengganu', 'Malaysia', 'Asia', 5.3290, 103.1408, 0),
    ('Vancouver', 'Vancouver', 'Canada', 'North America', 49.2827, -123.1207, 0)
    ]

'''
Cape Town: Often suggested for African representation in F1; great scenery and infrastructure.
Seoul: Huge tech hub with existing motorsport interest, could tie well with the Korean automotive industry.
Buenos Aires: Historic F1 location, potential for a return.
Chicago: Fills the gap in central U.S. with a global city profile.
Auckland: Expands Oceania representation beyond Australia.
Riyadh: Already hosts Formula E; could evolve into full F1 venue.
Lagos: Major economic center in Africa; bold long-term candidate.
Oslo: Scandinavia has no F1 GP currently — great fan base.
Kuala Terengganu: A hypothetical Malaysian alternative to Sepang.
Vancouver: Canada\'s west coast city with scenic urban race potential.
'''

# Insert unique circuit data into the fone_geography table
cursor.executemany('''
INSERT INTO fone_geography (circuit, city, country, continent, latitude, longitude, existing)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', geography_data)

# Add a geo_id column to the fone_calendar table and establish the foreign key relationship
cursor.execute('ALTER TABLE fone_calendar ADD COLUMN geo_id INTEGER')

# Update geo_id using the same_circuits dictionary
for key, values in same_circuits.items():
    cursor.execute('''
    UPDATE fone_calendar
    SET geo_id = (
        SELECT id FROM fone_geography
        WHERE fone_geography.circuit = ?
    )
    WHERE fone_calendar.circuit IN ({placeholders})
    '''.format(placeholders=','.join('?' * len(values))), [key] + values)

# Update geo_id for circuits that directly match
cursor.execute('''
UPDATE fone_calendar
SET geo_id = (
    SELECT id FROM fone_geography
    WHERE fone_geography.circuit = fone_calendar.circuit
)
WHERE geo_id IS NULL
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("fone_geography table created and linked to fone_calendar successfully.")

fone_geography table created and linked to fone_calendar successfully.


In [14]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the travel_logistic table
cursor.execute('''
CREATE TABLE IF NOT EXISTS travel_logistic (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    from_id INTEGER NOT NULL,
    from_circuit TEXT NOT NULL,
    to_id INTEGER NOT NULL,
    to_circuit TEXT NOT NULL,
    distance_km REAL,
    transport_mode TEXT
)
''')

# Fetch all circuits from the fone_geography table
cursor.execute('SELECT id, circuit FROM fone_geography')
circuits = cursor.fetchall()

# Generate all possible combinations of circuits
travel_data = []
for from_circuit in circuits:
    for to_circuit in circuits:
        if from_circuit[0] != to_circuit[0]:  # Avoid same circuit combinations
            travel_data.append((from_circuit[0], from_circuit[1], to_circuit[0], to_circuit[1]))

# Insert the combinations into the travel_logistic table
cursor.executemany('''
INSERT INTO travel_logistic (from_id, from_circuit, to_id, to_circuit)
VALUES (?, ?, ?, ?)
''', travel_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"{len(travel_data)} records inserted into the travel_logistic table.")

2256 records inserted into the travel_logistic table.


In [15]:
import math

# Function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Fetch latitude and longitude for all circuits
cursor.execute('SELECT id, latitude, longitude FROM fone_geography')
geo_data = {row[0]: (row[1], row[2]) for row in cursor.fetchall()}

# Update the distance_km column in the travel_logistic table
for from_id, from_circuit, to_id, to_circuit in travel_data:
    if from_id in geo_data and to_id in geo_data:
        lat1, lon1 = geo_data[from_id]
        lat2, lon2 = geo_data[to_id]
        distance = haversine(lat1, lon1, lat2, lon2)
        cursor.execute('''
        UPDATE travel_logistic
        SET distance_km = ?
        WHERE from_id = ? AND to_id = ?
        ''', (distance, from_id, to_id))

# Commit changes and close the connection
conn.commit()
conn.close()

print("Distances calculated and updated successfully.")

Distances calculated and updated successfully.
